# Module 4：RNN / LSTM / GRU 與序列資料

## 學習目標

1. 理解為什麼需要專門處理序列資料的模型
2. 掌握 RNN 的基本結構與梯度消失問題
3. 理解 LSTM 和 GRU 如何解決長期依賴問題
4. 學會處理文字資料：tokenization, embedding
5. 實作：情感分析、字元級語言模型

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import string
import random

# 設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

plt.rcParams['figure.figsize'] = (12, 4)

---

## Part 1：為什麼需要序列模型？

### 1.1 序列資料的特性

很多資料天生就是「有順序」的：
- **文字**：「我愛你」和「你愛我」意思完全不同
- **語音**：聲音訊號是時間序列
- **股價**：今天的價格跟昨天有關
- **影片**：連續的影格

### 1.2 MLP/CNN 處理序列的問題

1. **固定長度輸入**：句子長度不一，怎麼辦？
2. **沒有位置概念**：打亂順序結果一樣
3. **無法共享時間步的學習**：每個位置要獨立學習

In [ ]:
# 序列資料的例子

print("序列資料的例子：")
print("")
print("1. 文字序列：")
print("   '今天天氣真好' -> ['今', '天', '天', '氣', '真', '好']")
print("   'I love deep learning' -> ['I', 'love', 'deep', 'learning']")
print("")
print("2. 時間序列：")
print("   股價: [100, 102, 99, 105, 103, ...]")
print("   溫度: [20.5, 21.0, 22.3, 21.8, ...]")
print("")
print("3. 序列標註：")
print("   詞性標註: ['I/PRP', 'love/VBP', 'dogs/NNS']")
print("   命名實體: ['[PER]John', 'works', 'at', '[ORG]Google']")

---

## Part 2：循環神經網路 (RNN)

### 2.1 RNN 的核心思想

**讓網路有「記憶」**：把前一時刻的隱藏狀態傳給下一時刻

```
    h₀ ──→ h₁ ──→ h₂ ──→ h₃ ──→ h₄
    ↑      ↑      ↑      ↑      ↑
    x₀     x₁     x₂     x₃     x₄
    
    hₜ = tanh(Wₕₕ·hₜ₋₁ + Wₓₕ·xₜ + b)
```

**關鍵：** 同一組權重 (Wₕₕ, Wₓₕ) 在所有時間步共享！

In [ ]:
# 手動實現簡單的 RNN

class SimpleRNN(nn.Module):
    """最簡單的 RNN 實現"""
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        
        # 權重矩陣
        self.W_xh = nn.Linear(input_size, hidden_size)   # 輸入到隱藏
        self.W_hh = nn.Linear(hidden_size, hidden_size)  # 隱藏到隱藏
    
    def forward(self, x, h_prev=None):
        """
        x: (batch, seq_len, input_size)
        h_prev: (batch, hidden_size) 或 None
        """
        batch_size, seq_len, _ = x.shape
        
        # 初始化隱藏狀態
        if h_prev is None:
            h_prev = torch.zeros(batch_size, self.hidden_size, device=x.device)
        
        # 儲存每個時間步的隱藏狀態
        outputs = []
        h = h_prev
        
        # 對每個時間步
        for t in range(seq_len):
            x_t = x[:, t, :]  # 取出時間步 t 的輸入
            h = torch.tanh(self.W_xh(x_t) + self.W_hh(h))
            outputs.append(h)
        
        # 堆疊所有時間步的輸出
        outputs = torch.stack(outputs, dim=1)  # (batch, seq_len, hidden)
        
        return outputs, h

# 測試
rnn = SimpleRNN(input_size=10, hidden_size=20)
x = torch.randn(2, 5, 10)  # batch=2, seq_len=5, input=10
outputs, h_final = rnn(x)

print(f"輸入 shape: {x.shape}")
print(f"所有時間步的輸出 shape: {outputs.shape}")
print(f"最終隱藏狀態 shape: {h_final.shape}")

In [ ]:
# 使用 PyTorch 內建的 RNN

rnn_pytorch = nn.RNN(
    input_size=10,
    hidden_size=20,
    num_layers=1,
    batch_first=True  # 輸入格式 (batch, seq, feature)
)

x = torch.randn(2, 5, 10)
outputs, h_n = rnn_pytorch(x)

print(f"PyTorch RNN 輸出 shape: {outputs.shape}")
print(f"PyTorch RNN h_n shape: {h_n.shape}")

# 注意：h_n 的 shape 是 (num_layers, batch, hidden)

### 2.2 RNN 的梯度消失問題

**問題：** 當序列很長時，早期時間步的梯度會「消失」

**原因：** 反向傳播時，梯度要乘以 Wₕₕ 很多次
- 如果 |Wₕₕ| < 1：梯度越乘越小（消失）
- 如果 |Wₕₕ| > 1：梯度越乘越大（爆炸）

In [ ]:
# 梯度消失視覺化

def simulate_gradient_flow(weight_magnitude, seq_length):
    """模擬梯度在時間上的傳播"""
    gradients = [1.0]  # 從最後一層開始，梯度為 1
    for t in range(seq_length):
        gradients.append(gradients[-1] * weight_magnitude)
    return gradients[::-1]  # 反轉，讓 t=0 在前面

seq_length = 50
weight_magnitudes = [0.5, 0.9, 1.0, 1.1]

plt.figure(figsize=(12, 5))

for w in weight_magnitudes:
    grads = simulate_gradient_flow(w, seq_length)
    plt.plot(grads, label=f'|W| = {w}')

plt.xlabel('Time Step')
plt.ylabel('Relative Gradient Magnitude')
plt.title('Gradient Flow Through Time in RNN')
plt.legend()
plt.yscale('log')
plt.grid(True)
plt.show()

print("觀察：")
print("- |W| < 1：梯度指數衰減（消失）")
print("- |W| > 1：梯度指數增長（爆炸）")
print("- |W| = 1：理想情況，但實際很難達到")

---

## Part 3：LSTM (Long Short-Term Memory)

### 3.1 LSTM 的核心思想

**解決方案：** 引入「門控機制」(Gates) 來控制資訊流

LSTM 有三個門和一個細胞狀態 (cell state)：

1. **遺忘門 (Forget Gate)**：決定要「忘記」多少舊資訊
2. **輸入門 (Input Gate)**：決定要「記住」多少新資訊
3. **輸出門 (Output Gate)**：決定要「輸出」多少資訊
4. **細胞狀態 (Cell State)**：像一條「傳送帶」，讓資訊可以長距離流動

In [ ]:
# LSTM 結構圖解

print("""
LSTM Cell 結構：

                    ┌──────────────────────────────────────────┐
                    │           Cell State (Cₜ₋₁ → Cₜ)         │
                    │  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  │
                    │        ↑           ↑           ↓        │
                    │       ×           +           ×         │
                    │        ↑           ↑           ↓        │
                    │    ┌───────┐   ┌───────┐   ┌───────┐   │
                    │    │Forget │   │ Input │   │Output │   │
                    │    │ Gate  │   │ Gate  │   │ Gate  │   │
                    │    │  fₜ   │   │  iₜ   │   │  oₜ   │   │
                    │    └───┬───┘   └───┬───┘   └───┬───┘   │
                    │        │           │           │        │
                    │        └───────────┼───────────┘        │
                    │                    │                    │
                    │              [hₜ₋₁, xₜ]                 │
                    │                    ↑                    │
                    └────────────────────┼────────────────────┘
                                         │
                              xₜ ────────┴──────── hₜ₋₁

公式：
  fₜ = σ(Wf·[hₜ₋₁, xₜ] + bf)     # 遺忘門
  iₜ = σ(Wi·[hₜ₋₁, xₜ] + bi)     # 輸入門  
  C̃ₜ = tanh(Wc·[hₜ₋₁, xₜ] + bc)  # 候選細胞狀態
  Cₜ = fₜ × Cₜ₋₁ + iₜ × C̃ₜ       # 新細胞狀態
  oₜ = σ(Wo·[hₜ₋₁, xₜ] + bo)     # 輸出門
  hₜ = oₜ × tanh(Cₜ)             # 隱藏狀態
""")

In [ ]:
# 手動實現 LSTM Cell

class LSTMCell(nn.Module):
    """手動實現的 LSTM Cell"""
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        
        # 四個門的權重（合併成一個大矩陣更高效）
        self.gates = nn.Linear(input_size + hidden_size, 4 * hidden_size)
    
    def forward(self, x, states):
        """
        x: (batch, input_size)
        states: (h, c) 各為 (batch, hidden_size)
        """
        h_prev, c_prev = states
        
        # 合併輸入和前一個隱藏狀態
        combined = torch.cat([x, h_prev], dim=1)
        
        # 計算四個門（一次算完更高效）
        gates = self.gates(combined)
        
        # 分割成四個部分
        i, f, g, o = gates.chunk(4, dim=1)
        
        # 應用激活函數
        i = torch.sigmoid(i)  # 輸入門
        f = torch.sigmoid(f)  # 遺忘門
        g = torch.tanh(g)     # 候選細胞狀態
        o = torch.sigmoid(o)  # 輸出門
        
        # 更新細胞狀態
        c = f * c_prev + i * g
        
        # 計算隱藏狀態
        h = o * torch.tanh(c)
        
        return h, (h, c)

# 測試
cell = LSTMCell(input_size=10, hidden_size=20)
x = torch.randn(2, 10)  # batch=2, input=10
h = torch.zeros(2, 20)
c = torch.zeros(2, 20)

h_new, (h_out, c_out) = cell(x, (h, c))
print(f"輸入 shape: {x.shape}")
print(f"新的 h shape: {h_out.shape}")
print(f"新的 c shape: {c_out.shape}")

In [ ]:
# 使用 PyTorch 的 LSTM

lstm = nn.LSTM(
    input_size=10,
    hidden_size=20,
    num_layers=2,       # 堆疊 2 層 LSTM
    batch_first=True,
    dropout=0.2,        # 層間 dropout
    bidirectional=False # 單向
)

x = torch.randn(2, 5, 10)  # (batch, seq_len, input_size)
outputs, (h_n, c_n) = lstm(x)

print(f"輸入 shape: {x.shape}")
print(f"輸出 shape: {outputs.shape}")
print(f"h_n shape: {h_n.shape}  # (num_layers, batch, hidden)")
print(f"c_n shape: {c_n.shape}  # (num_layers, batch, hidden)")

### 3.2 LSTM 為什麼能解決梯度消失？

**關鍵：** Cell State 的更新是**加法**，不是乘法！

$$C_t = f_t \times C_{t-1} + i_t \times \tilde{C}_t$$

- 當 $f_t \approx 1$（不忘記）時，$C_{t-1}$ 幾乎原封不動地傳遞
- 梯度可以沿著 Cell State 這條「高速公路」直接流回早期時間步

---

## Part 4：GRU (Gated Recurrent Unit)

### 4.1 GRU 的結構

GRU 是 LSTM 的簡化版，只有兩個門：

1. **重置門 (Reset Gate)**：決定要忽略多少過去的資訊
2. **更新門 (Update Gate)**：決定要保留多少過去的資訊

In [ ]:
# GRU 結構圖解

print("""
GRU Cell 結構：

公式：
  rₜ = σ(Wr·[hₜ₋₁, xₜ])           # 重置門
  zₜ = σ(Wz·[hₜ₋₁, xₜ])           # 更新門
  h̃ₜ = tanh(W·[rₜ × hₜ₋₁, xₜ])    # 候選隱藏狀態
  hₜ = (1 - zₜ) × hₜ₋₁ + zₜ × h̃ₜ  # 新隱藏狀態

比較：
┌─────────────┬─────────────┬─────────────┐
│             │    LSTM     │     GRU     │
├─────────────┼─────────────┼─────────────┤
│ 門的數量    │      3      │      2      │
│ 狀態數量    │   2 (h, c)  │   1 (h)     │
│ 參數量      │     較多    │    較少     │
│ 計算速度    │     較慢    │    較快     │
│ 效果        │   通常較好  │   差不多    │
└─────────────┴─────────────┴─────────────┘
""")

In [ ]:
# 使用 PyTorch 的 GRU

gru = nn.GRU(
    input_size=10,
    hidden_size=20,
    num_layers=2,
    batch_first=True,
    dropout=0.2
)

x = torch.randn(2, 5, 10)
outputs, h_n = gru(x)

print(f"GRU 輸出 shape: {outputs.shape}")
print(f"GRU h_n shape: {h_n.shape}")
print("\n注意：GRU 只有 h_n，沒有 c_n（因為沒有 cell state）")

---

## Part 5：文字資料處理

### 5.1 Tokenization

把文字轉成數字的過程：
1. **字元級 (Character-level)**：每個字元一個 token
2. **詞級 (Word-level)**：每個詞一個 token
3. **子詞級 (Subword-level)**：BPE, WordPiece 等（Transformer 常用）

In [ ]:
# 簡單的詞級 tokenization

class SimpleTokenizer:
    """簡單的詞級分詞器"""
    def __init__(self):
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        self.vocab_size = 2
    
    def build_vocab(self, texts, min_freq=1):
        """建立詞彙表"""
        counter = Counter()
        for text in texts:
            words = text.lower().split()
            counter.update(words)
        
        for word, freq in counter.items():
            if freq >= min_freq and word not in self.word2idx:
                idx = len(self.word2idx)
                self.word2idx[word] = idx
                self.idx2word[idx] = word
        
        self.vocab_size = len(self.word2idx)
        print(f"Vocabulary size: {self.vocab_size}")
    
    def encode(self, text):
        """把文字轉成數字"""
        words = text.lower().split()
        return [self.word2idx.get(w, self.word2idx['<UNK>']) for w in words]
    
    def decode(self, indices):
        """把數字轉回文字"""
        return ' '.join([self.idx2word.get(i, '<UNK>') for i in indices])

# 測試
texts = [
    "I love deep learning",
    "Deep learning is amazing",
    "I enjoy studying neural networks"
]

tokenizer = SimpleTokenizer()
tokenizer.build_vocab(texts)

test_text = "I love neural networks"
encoded = tokenizer.encode(test_text)
decoded = tokenizer.decode(encoded)

print(f"\n原文: {test_text}")
print(f"編碼: {encoded}")
print(f"解碼: {decoded}")

### 5.2 Word Embedding

**問題：** 詞彙表中的索引是無意義的數字，無法表達詞與詞之間的關係

**解決方案：** 把每個詞映射到一個**稠密向量** (dense vector)

- 相似的詞會有相似的向量
- Embedding 可以是隨機初始化後訓練，或使用預訓練的（如 Word2Vec, GloVe）

In [ ]:
# Embedding 層

vocab_size = 100   # 詞彙表大小
embed_dim = 32     # 嵌入維度

embedding = nn.Embedding(vocab_size, embed_dim)

# 輸入是詞的索引
word_indices = torch.tensor([[1, 2, 3, 4],
                              [5, 6, 7, 0]])  # batch=2, seq_len=4

# 輸出是對應的嵌入向量
word_vectors = embedding(word_indices)

print(f"輸入 (詞索引) shape: {word_indices.shape}")
print(f"輸出 (詞向量) shape: {word_vectors.shape}")
print(f"\nEmbedding 權重 shape: {embedding.weight.shape}")
print("  = (vocab_size, embed_dim)")

In [ ]:
# Embedding + LSTM 的完整流程

class TextClassifier(nn.Module):
    """簡單的文本分類模型"""
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # *2 因為雙向
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # x: (batch, seq_len) - 詞索引
        
        # Embedding
        embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
        embedded = self.dropout(embedded)
        
        # LSTM
        lstm_out, (h_n, c_n) = self.lstm(embedded)
        # lstm_out: (batch, seq_len, hidden*2)
        # h_n: (2, batch, hidden) - 雙向所以是 2
        
        # 取最後一個時間步的輸出（或可以用 h_n）
        # 雙向 LSTM：拼接正向最後和反向最後
        hidden = torch.cat([h_n[-2], h_n[-1]], dim=1)  # (batch, hidden*2)
        
        # 分類
        out = self.fc(self.dropout(hidden))
        
        return out

# 測試
model = TextClassifier(vocab_size=1000, embed_dim=128, hidden_dim=64, num_classes=2)
x = torch.randint(0, 1000, (4, 20))  # batch=4, seq_len=20
out = model(x)

print(f"輸入 shape: {x.shape}")
print(f"輸出 shape: {out.shape}")

### 5.3 處理變長序列

**問題：** 不同句子長度不同，怎麼組成 batch？

**解決方案：**
1. **Padding**：用 0 填充到相同長度
2. **Pack/Pad**：PyTorch 的 `pack_padded_sequence` 讓 LSTM 忽略 padding

In [ ]:
# 處理變長序列

# 三個不同長度的序列
seq1 = torch.tensor([1, 2, 3, 4, 5])      # 長度 5
seq2 = torch.tensor([6, 7, 8])            # 長度 3
seq3 = torch.tensor([9, 10, 11, 12])      # 長度 4

sequences = [seq1, seq2, seq3]
lengths = torch.tensor([len(s) for s in sequences])

# Padding
padded = pad_sequence(sequences, batch_first=True, padding_value=0)
print(f"Padded sequences:\n{padded}")
print(f"Lengths: {lengths}")

In [ ]:
# 使用 pack_padded_sequence

# 注意：需要先按長度排序（降序）
lengths_sorted, sort_idx = lengths.sort(descending=True)
padded_sorted = padded[sort_idx]

print(f"排序後的序列:\n{padded_sorted}")
print(f"排序後的長度: {lengths_sorted}")

# 模擬 embedding 後的資料
embed_dim = 4
embedded = torch.randn(3, 5, embed_dim)  # (batch, max_len, embed_dim)

# Pack
packed = pack_padded_sequence(embedded, lengths_sorted.cpu(), 
                               batch_first=True, enforce_sorted=True)

print(f"\nPacked data shape: {packed.data.shape}")
print(f"Packed batch_sizes: {packed.batch_sizes}")

---

## Part 6：實作 - 情感分析

使用 LSTM 來判斷電影評論是正面還是負面。

In [ ]:
# 建立簡單的情感分析資料集

# 正面評論
positive_texts = [
    "This movie is great and amazing",
    "I love this film so much",
    "Excellent acting and wonderful story",
    "Best movie I have ever seen",
    "Fantastic film with great performances",
    "This is absolutely brilliant",
    "Amazing movie highly recommended",
    "Wonderful experience great movie",
    "I really enjoyed this film",
    "Perfect movie loved every moment",
    "Outstanding performance by all actors",
    "This film exceeded my expectations",
    "Beautiful story and amazing visuals",
    "One of the best films ever",
    "Incredible movie must watch",
]

# 負面評論
negative_texts = [
    "This movie is terrible and boring",
    "I hate this film completely",
    "Worst movie I have ever seen",
    "Bad acting and terrible story",
    "Complete waste of time",
    "This is absolutely awful",
    "Boring movie do not watch",
    "Terrible experience bad movie",
    "I really hated this film",
    "Worst film avoid at all costs",
    "Disappointing and poorly made",
    "This movie was so bad",
    "Horrible story and bad acting",
    "One of the worst films ever",
    "Awful movie not recommended",
]

# 建立資料集
all_texts = positive_texts + negative_texts
all_labels = [1] * len(positive_texts) + [0] * len(negative_texts)

print(f"正面評論數量: {len(positive_texts)}")
print(f"負面評論數量: {len(negative_texts)}")

In [ ]:
# 建立詞彙表和資料集

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=50):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Tokenize
        tokens = self.tokenizer.encode(text)
        
        # Padding 或 截斷
        if len(tokens) < self.max_len:
            tokens = tokens + [0] * (self.max_len - len(tokens))
        else:
            tokens = tokens[:self.max_len]
        
        return {
            'input_ids': torch.tensor(tokens),
            'label': torch.tensor(label)
        }

# 建立 tokenizer
tokenizer = SimpleTokenizer()
tokenizer.build_vocab(all_texts)

# 分割訓練和測試
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    all_texts, all_labels, test_size=0.2, random_state=42
)

train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
test_dataset = SentimentDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

print(f"訓練集: {len(train_dataset)}")
print(f"測試集: {len(test_dataset)}")

In [ ]:
# 定義模型

class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            embed_dim, hidden_dim, 
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.3 if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim * 2, 1)  # 二分類
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # x: (batch, seq_len)
        embedded = self.dropout(self.embedding(x))
        
        # LSTM
        lstm_out, (h_n, c_n) = self.lstm(embedded)
        
        # 取雙向 LSTM 的最後隱藏狀態
        hidden = torch.cat([h_n[-2], h_n[-1]], dim=1)
        
        # 分類
        out = self.fc(self.dropout(hidden))
        return out.squeeze(1)

# 建立模型
model = SentimentLSTM(
    vocab_size=tokenizer.vocab_size,
    embed_dim=64,
    hidden_dim=32,
    num_layers=1
).to(device)

print(model)
print(f"\n參數量: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 訓練

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
history = {'train_loss': [], 'train_acc': [], 'test_acc': []}

for epoch in range(num_epochs):
    # 訓練
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].float().to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).long()
        correct += (preds == labels.long()).sum().item()
        total += labels.size(0)
    
    train_acc = 100 * correct / total
    history['train_loss'].append(total_loss / len(train_loader))
    history['train_acc'].append(train_acc)
    
    # 測試
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids)
            preds = (torch.sigmoid(outputs) > 0.5).long()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    test_acc = 100 * correct / total
    history['test_acc'].append(test_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {history['train_loss'][-1]:.4f}, "
              f"Train Acc: {train_acc:.1f}%, Test Acc: {test_acc:.1f}%")

In [ ]:
# 視覺化

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'])
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True)

axes[1].plot(history['train_acc'], label='Train')
axes[1].plot(history['test_acc'], label='Test')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 測試新評論

def predict_sentiment(text, model, tokenizer):
    model.eval()
    tokens = tokenizer.encode(text)
    # Padding
    tokens = tokens + [0] * (50 - len(tokens)) if len(tokens) < 50 else tokens[:50]
    input_ids = torch.tensor([tokens]).to(device)
    
    with torch.no_grad():
        output = model(input_ids)
        prob = torch.sigmoid(output).item()
    
    sentiment = "Positive" if prob > 0.5 else "Negative"
    return sentiment, prob

# 測試一些新評論
test_reviews = [
    "This movie is absolutely wonderful",
    "I hated every minute of this film",
    "Great story and amazing acting",
    "Terrible movie waste of time",
    "The movie was okay not great"
]

print("情感分析結果：")
print("-" * 60)
for review in test_reviews:
    sentiment, prob = predict_sentiment(review, model, tokenizer)
    print(f"'{review}'")
    print(f"  -> {sentiment} (confidence: {prob:.2%})")
    print()

---

## Part 7：實作 - 字元級語言模型

訓練一個模型來預測下一個字元，可以用來生成文字。

In [ ]:
# 準備資料

# 使用簡單的訓練文本
text = """To be or not to be that is the question
Whether tis nobler in the mind to suffer
The slings and arrows of outrageous fortune
Or to take arms against a sea of troubles
And by opposing end them To die to sleep
No more and by a sleep to say we end
The heartache and the thousand natural shocks
That flesh is heir to Tis a consummation
Devoutly to be wished To die to sleep
To sleep perchance to dream ay theres the rub"""

# 建立字元到索引的映射
chars = sorted(list(set(text)))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for i, c in enumerate(chars)}
vocab_size = len(chars)

print(f"文本長度: {len(text)} 字元")
print(f"詞彙表大小: {vocab_size}")
print(f"字元: {chars}")

In [ ]:
# 建立訓練資料

def create_sequences(text, seq_length):
    """把文本分成固定長度的序列"""
    sequences = []
    targets = []
    
    for i in range(len(text) - seq_length):
        seq = text[i:i + seq_length]
        target = text[i + 1:i + seq_length + 1]
        
        seq_idx = [char2idx[c] for c in seq]
        target_idx = [char2idx[c] for c in target]
        
        sequences.append(seq_idx)
        targets.append(target_idx)
    
    return torch.tensor(sequences), torch.tensor(targets)

seq_length = 50
X, y = create_sequences(text, seq_length)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\n範例輸入: '{text[:seq_length]}'")
print(f"範例目標: '{text[1:seq_length+1]}'")

In [ ]:
# 定義字元級語言模型

class CharLM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=2):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(
            embed_dim, hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden=None):
        # x: (batch, seq_len)
        embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
        
        if hidden is None:
            lstm_out, hidden = self.lstm(embedded)
        else:
            lstm_out, hidden = self.lstm(embedded, hidden)
        
        # 每個位置都預測下一個字元
        out = self.fc(lstm_out)  # (batch, seq_len, vocab_size)
        
        return out, hidden
    
    def init_hidden(self, batch_size, device):
        h = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        c = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        return (h, c)

# 建立模型
model = CharLM(
    vocab_size=vocab_size,
    embed_dim=64,
    hidden_dim=128,
    num_layers=2
).to(device)

print(model)
print(f"\n參數量: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 訓練

# 建立 DataLoader
dataset = torch.utils.data.TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

num_epochs = 200
losses = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch_x, batch_y in loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        
        outputs, _ = model(batch_x)
        # outputs: (batch, seq_len, vocab_size)
        # 需要 reshape 成 (batch*seq_len, vocab_size)
        loss = criterion(outputs.view(-1, vocab_size), batch_y.view(-1))
        
        loss.backward()
        
        # 梯度裁剪，防止梯度爆炸
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(loader)
    losses.append(avg_loss)
    
    if (epoch + 1) % 40 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f}")

# 視覺化
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Character LM Training Loss')
plt.grid(True)
plt.show()

In [ ]:
# 生成文字

def generate_text(model, start_text, length=200, temperature=1.0):
    """
    從 start_text 開始生成文字
    temperature: 控制隨機性，越低越確定，越高越隨機
    """
    model.eval()
    
    # 把起始文字轉成索引
    chars_idx = [char2idx.get(c, 0) for c in start_text]
    input_seq = torch.tensor([chars_idx]).to(device)
    
    hidden = model.init_hidden(1, device)
    generated = start_text
    
    with torch.no_grad():
        # 先處理起始文字，更新 hidden state
        _, hidden = model(input_seq, hidden)
        
        # 生成新字元
        current_char = chars_idx[-1]
        
        for _ in range(length):
            input_seq = torch.tensor([[current_char]]).to(device)
            output, hidden = model(input_seq, hidden)
            
            # 取最後一個位置的輸出
            logits = output[0, -1] / temperature
            probs = F.softmax(logits, dim=0)
            
            # 採樣下一個字元
            next_char_idx = torch.multinomial(probs, 1).item()
            next_char = idx2char[next_char_idx]
            
            generated += next_char
            current_char = next_char_idx
    
    return generated

# 測試不同 temperature
print("=" * 60)
print("Temperature = 0.5 (較確定):")
print("=" * 60)
print(generate_text(model, "To be or not to be", length=150, temperature=0.5))

print("\n" + "=" * 60)
print("Temperature = 1.0 (平衡):")
print("=" * 60)
print(generate_text(model, "To be or not to be", length=150, temperature=1.0))

print("\n" + "=" * 60)
print("Temperature = 1.5 (較隨機):")
print("=" * 60)
print(generate_text(model, "To be or not to be", length=150, temperature=1.5))

---

## 練習題（已完成，請閱讀理解）

### 練習 1：比較 RNN, LSTM, GRU

**目標：** 在相同任務上比較三種模型的表現

**Hint：** LSTM 和 GRU 通常比普通 RNN 好，特別是長序列

In [ ]:
# 練習 1：比較三種 RNN 變體

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, rnn_type='lstm'):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        if rnn_type == 'rnn':
            self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        elif rnn_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        
        self.rnn_type = rnn_type
        self.fc = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        embedded = self.embedding(x)
        
        if self.rnn_type == 'lstm':
            _, (h_n, _) = self.rnn(embedded)
        else:
            _, h_n = self.rnn(embedded)
        
        out = self.fc(h_n[-1])
        return out.squeeze(1)

def train_model_quick(model, train_loader, epochs=30):
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    losses = []
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].float().to(device)
            
            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()
        losses.append(total_loss / len(train_loader))
    
    return losses

# 訓練三種模型
results = {}
for rnn_type in ['rnn', 'lstm', 'gru']:
    print(f"Training {rnn_type.upper()}...")
    torch.manual_seed(42)
    model = RNNModel(tokenizer.vocab_size, 64, 32, rnn_type)
    losses = train_model_quick(model, train_loader, epochs=30)
    results[rnn_type] = losses

# 視覺化
plt.figure(figsize=(10, 5))
for name, losses in results.items():
    plt.plot(losses, label=name.upper())
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('RNN vs LSTM vs GRU')
plt.legend()
plt.grid(True)
plt.show()

print("\n觀察：LSTM 和 GRU 通常比普通 RNN 收斂更穩定")

### 練習 2：雙向 LSTM (Bidirectional LSTM)

**目標：** 理解雙向 LSTM 的優勢

**Hint：** 雙向 LSTM 可以同時看到「過去」和「未來」的上下文

In [ ]:
# 練習 2：雙向 LSTM

print("""
雙向 LSTM 結構：

Forward:   h₁ → h₂ → h₃ → h₄
            ↑    ↑    ↑    ↑
Input:     x₁   x₂   x₃   x₄
            ↓    ↓    ↓    ↓
Backward:  h₁ ← h₂ ← h₃ ← h₄

每個位置的輸出 = [forward_h, backward_h]

優勢：
- 對於分類任務，可以看到完整的上下文
- 填空任務："I love _____ and cats" -> "dogs"

限制：
- 不能用於生成任務（因為看不到未來）
""")

# 單向 vs 雙向
lstm_uni = nn.LSTM(10, 20, batch_first=True, bidirectional=False)
lstm_bi = nn.LSTM(10, 20, batch_first=True, bidirectional=True)

x = torch.randn(2, 5, 10)

out_uni, (h_uni, _) = lstm_uni(x)
out_bi, (h_bi, _) = lstm_bi(x)

print(f"單向 LSTM 輸出 shape: {out_uni.shape}")
print(f"雙向 LSTM 輸出 shape: {out_bi.shape}")
print(f"\n單向 h_n shape: {h_uni.shape}")
print(f"雙向 h_n shape: {h_bi.shape}")

### 練習 3：Attention 機制預覽

**目標：** 理解為什麼需要 Attention，為 Module 5 做準備

**Hint：** RNN 的瓶頸是把整個序列壓縮成一個固定向量

In [ ]:
# 練習 3：簡單的 Attention

class AttentionLSTM(nn.Module):
    """帶有簡單 Attention 的 LSTM"""
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        
        # Attention 層
        self.attention = nn.Linear(hidden_dim * 2, 1)
        
        self.fc = nn.Linear(hidden_dim * 2, 1)
    
    def forward(self, x):
        # x: (batch, seq_len)
        embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
        
        # LSTM 輸出所有時間步
        lstm_out, _ = self.lstm(embedded)  # (batch, seq_len, hidden*2)
        
        # 計算 attention weights
        attention_scores = self.attention(lstm_out).squeeze(-1)  # (batch, seq_len)
        attention_weights = F.softmax(attention_scores, dim=1)  # (batch, seq_len)
        
        # 加權平均
        # (batch, seq_len, 1) * (batch, seq_len, hidden*2) -> sum over seq_len
        context = torch.sum(attention_weights.unsqueeze(-1) * lstm_out, dim=1)  # (batch, hidden*2)
        
        # 分類
        out = self.fc(context)
        
        return out.squeeze(1), attention_weights

# 測試
model = AttentionLSTM(tokenizer.vocab_size, 64, 32)
x = torch.randint(0, tokenizer.vocab_size, (2, 20))
out, attn_weights = model(x)

print(f"輸出 shape: {out.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"\nAttention weights (第一個樣本): {attn_weights[0].detach().numpy()[:10]}...")
print(f"Attention weights 總和: {attn_weights[0].sum().item():.4f} (應該接近 1.0)")

In [ ]:
# 視覺化 Attention

# 使用訓練好的情感分析模型
model = AttentionLSTM(tokenizer.vocab_size, 64, 32).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 快速訓練
for epoch in range(30):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].float().to(device)
        
        optimizer.zero_grad()
        outputs, _ = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# 測試並視覺化 attention
test_sentence = "This movie is absolutely wonderful and amazing"
tokens = tokenizer.encode(test_sentence)
tokens = tokens + [0] * (50 - len(tokens))
input_ids = torch.tensor([tokens]).to(device)

model.eval()
with torch.no_grad():
    output, attn = model(input_ids)

prob = torch.sigmoid(output).item()
words = test_sentence.lower().split()
attn_values = attn[0, :len(words)].cpu().numpy()

# 視覺化
plt.figure(figsize=(12, 4))
plt.bar(range(len(words)), attn_values)
plt.xticks(range(len(words)), words, rotation=45)
plt.xlabel('Words')
plt.ylabel('Attention Weight')
plt.title(f'Attention Visualization\nPrediction: {"Positive" if prob > 0.5 else "Negative"} ({prob:.2%})')
plt.tight_layout()
plt.show()

print("觀察：模型應該會對情感詞（如 'wonderful', 'amazing'）給予更高的 attention")

---

## Part 8：進階序列建模技巧

### 8.1 Teacher Forcing vs Scheduled Sampling

**Teacher Forcing：** 訓練時使用真實的前一個 token 作為輸入（而非模型預測的）

**問題：** 訓練和推論時的分佈不同 (Exposure Bias)

**解決方案：** Scheduled Sampling - 逐漸從 teacher forcing 過渡到自由生成

---

## Module 4 總結

### 核心概念

1. **RNN 基礎**：
   - 隱藏狀態在時間步之間傳遞
   - 權重在所有時間步共享
   - 問題：梯度消失/爆炸

2. **LSTM**：
   - 三個門：遺忘門、輸入門、輸出門
   - Cell State：資訊的「高速公路」
   - 解決長期依賴問題

3. **GRU**：
   - LSTM 的簡化版
   - 兩個門：重置門、更新門
   - 參數更少，速度更快

4. **文字處理**：
   - Tokenization：文字 → 數字
   - Embedding：索引 → 稠密向量
   - Padding：處理變長序列

5. **實用技巧**：
   - 雙向 LSTM：同時看過去和未來
   - 梯度裁剪：防止梯度爆炸
   - Attention：讓模型「關注」重要部分

### 下一步：Module 5 - Attention 與 Transformer

In [ ]:
# Teacher Forcing 示範

print("""
Teacher Forcing 比較：

訓練時 (Teacher Forcing = True)：
  輸入: <start> I    love  NLP
  目標:  I      love  NLP   <end>
  ↓ 每一步都用真實的前一個詞

訓練時 (Teacher Forcing = False)：
  輸入: <start> → 模型預測 → 模型預測 → ...
  ↓ 每一步都用自己預測的詞（可能會累積錯誤）

推論時：
  永遠是 Free Running（沒有真實標籤可用）
""")

class TeacherForcingDemo(nn.Module):
    """展示 Teacher Forcing 的語言模型"""
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.hidden_dim = hidden_dim
    
    def forward(self, x, targets=None, teacher_forcing_ratio=0.5):
        """
        x: (batch, seq_len) - 起始 token
        targets: (batch, seq_len) - 真實的目標序列
        teacher_forcing_ratio: 使用 teacher forcing 的機率
        """
        batch_size, seq_len = x.shape
        device = x.device
        
        # 初始化
        outputs = []
        h = torch.zeros(1, batch_size, self.hidden_dim, device=device)
        c = torch.zeros(1, batch_size, self.hidden_dim, device=device)
        
        # 第一個輸入
        current_input = x[:, 0:1]  # (batch, 1)
        
        for t in range(seq_len):
            embedded = self.embedding(current_input)
            lstm_out, (h, c) = self.lstm(embedded, (h, c))
            output = self.fc(lstm_out)  # (batch, 1, vocab_size)
            outputs.append(output)
            
            # 決定下一個輸入
            if targets is not None and t < seq_len - 1:
                use_teacher = random.random() < teacher_forcing_ratio
                if use_teacher:
                    current_input = targets[:, t:t+1]  # 用真實值
                else:
                    current_input = output.argmax(dim=-1)  # 用預測值
            elif t < seq_len - 1:
                current_input = output.argmax(dim=-1)
        
        return torch.cat(outputs, dim=1)

# 示範
print("Teacher Forcing Ratio 的影響：")
print("- ratio=1.0: 完全 teacher forcing（訓練快但可能有 exposure bias）")
print("- ratio=0.5: 一半一半（平衡）")
print("- ratio=0.0: 完全自由生成（接近推論時的情況）")

### 8.2 Beam Search 解碼

**Greedy Search**：每一步選機率最高的 token（可能錯過更好的整體序列）

**Beam Search**：保留 top-k 個候選序列，最後選總機率最高的

In [ ]:
# Beam Search 實現

def beam_search_decode(model, start_idx, idx2char, char2idx, beam_width=3, max_len=50):
    """
    Beam Search 文字生成
    
    beam_width: 同時保留的候選數量
    """
    model.eval()
    device = next(model.parameters()).device
    
    # 初始化 beam: (sequence, log_prob, hidden_state)
    hidden = model.init_hidden(1, device)
    beams = [([start_idx], 0.0, hidden)]
    
    completed = []
    
    for step in range(max_len):
        all_candidates = []
        
        for seq, score, hidden in beams:
            # 如果已經生成了終止符（這裡用換行符模擬）
            if len(seq) > 1 and seq[-1] == char2idx.get('\n', -1):
                completed.append((seq, score))
                continue
            
            # 準備輸入
            input_tensor = torch.tensor([[seq[-1]]], device=device)
            
            with torch.no_grad():
                output, new_hidden = model(input_tensor, hidden)
                log_probs = F.log_softmax(output[0, -1], dim=0)
            
            # 取 top-k 候選
            top_probs, top_indices = log_probs.topk(beam_width)
            
            for prob, idx in zip(top_probs, top_indices):
                new_seq = seq + [idx.item()]
                new_score = score + prob.item()
                all_candidates.append((new_seq, new_score, new_hidden))
        
        # 保留 top beam_width 個候選
        all_candidates.sort(key=lambda x: x[1], reverse=True)
        beams = all_candidates[:beam_width]
        
        if not beams:
            break
    
    # 結合 completed 和 beams，選最好的
    all_results = completed + [(s, sc) for s, sc, _ in beams]
    all_results.sort(key=lambda x: x[1] / len(x[0]), reverse=True)  # 長度正規化
    
    best_seq = all_results[0][0] if all_results else [start_idx]
    return ''.join([idx2char.get(i, '?') for i in best_seq])

# 比較 Greedy vs Beam Search
print("Greedy Search vs Beam Search 比較：")
print("=" * 50)

# Greedy (temperature=0 等價)
print("\nGreedy Search:")
greedy_text = generate_text(model, "To be", length=50, temperature=0.1)
print(greedy_text[:80])

# Beam Search
print("\nBeam Search (beam_width=3):")
start_chars = "To be"
start_indices = [char2idx[c] for c in start_chars]
# 由於我們的模型結構，這裡用簡化的展示
print("(Beam Search 會探索多個路徑，選擇整體機率最高的序列)")
print("(適合翻譯、摘要等需要高品質輸出的任務)")

### 8.3 Sequence-to-Sequence (Seq2Seq) 架構

**應用：** 機器翻譯、文字摘要、對話系統

**結構：** Encoder-Decoder
- **Encoder**：把輸入序列壓縮成一個「context vector」
- **Decoder**：從 context vector 生成輸出序列

In [ ]:
# Seq2Seq 架構實現

class Encoder(nn.Module):
    """Seq2Seq Encoder"""
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, 
                           batch_first=True, bidirectional=True)
        # 雙向 -> 單向的投影（給 decoder 用）
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)
    
    def forward(self, x):
        # x: (batch, src_len)
        embedded = self.embedding(x)
        outputs, (h_n, c_n) = self.lstm(embedded)
        
        # 把雙向的 hidden 合併
        # h_n: (num_layers*2, batch, hidden)
        h_n = torch.cat([h_n[-2], h_n[-1]], dim=1)  # (batch, hidden*2)
        c_n = torch.cat([c_n[-2], c_n[-1]], dim=1)
        
        # 投影到 decoder 的維度
        h_n = torch.tanh(self.fc(h_n)).unsqueeze(0)  # (1, batch, hidden)
        c_n = torch.tanh(self.fc(c_n)).unsqueeze(0)
        
        return outputs, (h_n, c_n)


class Decoder(nn.Module):
    """Seq2Seq Decoder"""
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden):
        # x: (batch, 1) - 一次一個 token
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)
        prediction = self.fc(output)
        return prediction, hidden


class Seq2Seq(nn.Module):
    """完整的 Seq2Seq 模型"""
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: (batch, src_len)
        # trg: (batch, trg_len)
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.fc.out_features
        
        # 儲存輸出
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size, device=self.device)
        
        # Encode
        _, hidden = self.encoder(src)
        
        # Decoder 的第一個輸入是 <sos> token（假設是 trg 的第一個）
        dec_input = trg[:, 0:1]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(dec_input, hidden)
            outputs[:, t] = output.squeeze(1)
            
            # Teacher Forcing
            use_teacher = random.random() < teacher_forcing_ratio
            dec_input = trg[:, t:t+1] if use_teacher else output.argmax(dim=-1)
        
        return outputs

# 建立 Seq2Seq 模型
src_vocab = 100
trg_vocab = 80
embed_dim = 64
hidden_dim = 128

encoder = Encoder(src_vocab, embed_dim, hidden_dim)
decoder = Decoder(trg_vocab, embed_dim, hidden_dim)
seq2seq = Seq2Seq(encoder, decoder, device).to(device)

# 測試
src = torch.randint(0, src_vocab, (2, 10)).to(device)  # 源語言
trg = torch.randint(0, trg_vocab, (2, 8)).to(device)   # 目標語言

output = seq2seq(src, trg, teacher_forcing_ratio=0.5)
print(f"Seq2Seq 架構：")
print(f"  源序列 shape: {src.shape}")
print(f"  目標序列 shape: {trg.shape}")
print(f"  輸出 shape: {output.shape}")
print(f"\n參數量: {sum(p.numel() for p in seq2seq.parameters()):,}")

### 8.4 梯度裁剪與 Masking 技巧

**梯度裁剪 (Gradient Clipping)**：防止 RNN 訓練時的梯度爆炸

**Masking**：正確處理 padding，避免 padding 影響計算

In [ ]:
# 梯度裁剪最佳實踐

def train_with_gradient_clipping(model, optimizer, criterion, x, y, max_grad_norm=1.0):
    """展示梯度裁剪的正確用法"""
    model.train()
    optimizer.zero_grad()
    
    output = model(x)
    loss = criterion(output.view(-1, output.size(-1)), y.view(-1))
    loss.backward()
    
    # 記錄裁剪前的梯度範數
    total_norm_before = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm_before += param_norm.item() ** 2
    total_norm_before = total_norm_before ** 0.5
    
    # 梯度裁剪
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    
    # 記錄裁剪後的梯度範數
    total_norm_after = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm_after += param_norm.item() ** 2
    total_norm_after = total_norm_after ** 0.5
    
    optimizer.step()
    
    return loss.item(), total_norm_before, total_norm_after

print("梯度裁剪策略比較：")
print("=" * 60)
print("""
1. clip_grad_norm_（L2 norm）：
   torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
   - 最常用，按整體梯度範數縮放
   - 保持梯度方向不變
   
2. clip_grad_value_（按值裁剪）：
   torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)
   - 每個梯度元素單獨裁剪
   - 可能改變梯度方向

推薦配置：
- RNN/LSTM: max_norm=1.0 到 5.0
- Transformer: max_norm=1.0
- 一般神經網路: 可能不需要（沒有長時間依賴）
""")

# 示範 Masking
print("\n" + "=" * 60)
print("Masking 示範：")
print("=" * 60)

# 建立帶 padding 的序列
seq1 = [1, 2, 3, 4, 5, 0, 0, 0]  # 長度 5，後面是 padding
seq2 = [6, 7, 8, 0, 0, 0, 0, 0]  # 長度 3
seq3 = [9, 10, 11, 12, 13, 14, 15, 16]  # 長度 8

batch = torch.tensor([seq1, seq2, seq3])
lengths = torch.tensor([5, 3, 8])

# 建立 mask
mask = torch.arange(batch.size(1)).expand(len(lengths), batch.size(1))
mask = mask < lengths.unsqueeze(1)

print(f"Batch:\n{batch}")
print(f"\nLengths: {lengths}")
print(f"\nMask:\n{mask.int()}")

# 在計算 loss 時使用 mask
def masked_cross_entropy(pred, target, mask):
    """忽略 padding 位置的 cross entropy"""
    # pred: (batch, seq_len, vocab_size)
    # target: (batch, seq_len)
    # mask: (batch, seq_len) - True 表示真實 token
    
    batch_size, seq_len, vocab_size = pred.shape
    
    # Flatten
    pred_flat = pred.view(-1, vocab_size)
    target_flat = target.view(-1)
    mask_flat = mask.view(-1)
    
    # 計算所有位置的 loss
    loss = F.cross_entropy(pred_flat, target_flat, reduction='none')
    
    # 只保留非 padding 位置
    loss = loss * mask_flat.float()
    
    # 平均（只除以非 padding 的數量）
    return loss.sum() / mask_flat.sum()

print("\n使用 mask 計算 loss 可以避免 padding 影響結果！")

---

## Module 4 完整總結

### 🎯 核心概念對比

| 模型 | 門數 | 狀態 | 參數量 | 適用場景 |
|------|------|------|--------|----------|
| **Simple RNN** | 0 | h | 少 | 短序列、簡單任務 |
| **LSTM** | 3 | h, c | 多 | 長序列、需要長期記憶 |
| **GRU** | 2 | h | 中 | 資源受限、速度優先 |

### 🔑 關鍵公式

```
RNN:
  hₜ = tanh(Wₓₕ·xₜ + Wₕₕ·hₜ₋₁ + b)

LSTM:
  fₜ = σ(Wf·[hₜ₋₁, xₜ] + bf)      # 遺忘門
  iₜ = σ(Wi·[hₜ₋₁, xₜ] + bi)      # 輸入門
  oₜ = σ(Wo·[hₜ₋₁, xₜ] + bo)      # 輸出門
  C̃ₜ = tanh(Wc·[hₜ₋₁, xₜ] + bc)   # 候選狀態
  Cₜ = fₜ ⊙ Cₜ₋₁ + iₜ ⊙ C̃ₜ        # 細胞狀態
  hₜ = oₜ ⊙ tanh(Cₜ)              # 隱藏狀態

GRU:
  rₜ = σ(Wr·[hₜ₋₁, xₜ])           # 重置門
  zₜ = σ(Wz·[hₜ₋₁, xₜ])           # 更新門
  h̃ₜ = tanh(W·[rₜ ⊙ hₜ₋₁, xₜ])    # 候選狀態
  hₜ = (1-zₜ) ⊙ hₜ₋₁ + zₜ ⊙ h̃ₜ   # 新狀態
```

### 🏗️ 架構演進

```
RNN (1980s)     → 最基礎，有梯度消失問題
LSTM (1997)     → 引入門控，解決長期依賴
GRU (2014)      → LSTM 簡化版，效果相當
Seq2Seq (2014)  → Encoder-Decoder，用於翻譯
Attention (2015)→ 動態關注，緩解瓶頸問題
Transformer (2017) → 完全捨棄 RNN，全 attention
```

### 💡 實務技巧

1. **模型選擇**：
   - 預設用 LSTM（最穩定）
   - 資源受限考慮 GRU
   - 序列 < 100 可考慮 RNN

2. **訓練技巧**：
   - **必須**使用梯度裁剪（max_norm=1.0~5.0）
   - 使用 pack_padded_sequence 處理變長序列
   - Dropout 只用在層之間，不用在時間步之間

3. **超參數建議**：
   - hidden_dim: 128~512
   - num_layers: 1~3（更深用殘差連接）
   - embed_dim: 100~300

4. **文字生成**：
   - Temperature 控制隨機性（0.7~1.0 較好）
   - Beam Search 生成更高品質的序列
   - 用 Teacher Forcing 加速訓練

5. **常見問題**：
   - 梯度爆炸 → 梯度裁剪
   - 過擬合 → Dropout + 早停
   - 訓練慢 → 減少層數或用 GRU

### 🚀 下一步

- Module 5：Attention 與 Transformer（徹底解決 RNN 的瓶頸問題）

---

**恭喜完成 RNN 模組！** 🎉

LSTM/GRU 是處理序列資料的經典方法，即使在 Transformer 時代，理解這些概念對於理解現代架構仍然非常重要。